In order to get some datasets I needed to export them by convertind the dataframe to a csv and then outputting the contents of the file to the terminal such as the example below.
```Python
# output the contents of the dataframe to CSV wile also removing the index for pushing it to a dataframe.
wards_altered.to_csv("wards_altered.csv", index=False)

# open the file and read the contents of the file to the terminal.
with open("wards_altered.csv", "r") as f:
    print(f.read())
```

# Comparing Strings

In [4]:
# install needed packages using pip
import sys
#If you have pip you can use this line
# !{sys.executable} -m pip install fuzzywuzzy
# If you have anaconda you can use this line
!conda install  -c conda-forge --yes  --prefix {sys.prefix} fuzzywuzzy

Solving environment: done

# All requested packages already installed.



In [48]:
# import packages
import pandas as pd
from fuzzywuzzy import process

# import data
restaurants = pd.read_csv('../datasets/restaurants_L2_dirty.csv')
restaurants.rename(columns={'type':'cuisine_type', 'name':'rest_name'}, inplace=True)
restaurants.head()

,Unnamed: 0,rest_name,addr,city,phone,cuisine_type
0,0,kokomo,6333 w. third st.,la,2139330773,american
1,1,feenix,8358 sunset blvd. west,hollywood,2138486677,american
2,2,parkway,510 s. arroyo pkwy .,pasadena,8187951001,californian
3,3,r-23,923 e. third st.,los angeles,2136877178,japanese
4,4,gumbo,6333 w. third st.,la,2139330358,cajun/creole


In [53]:
# drop the index that made it into the csv
try:
    restaurants = restaurants.drop('Unnamed: 0', axis=1)
except:
    "error while dropping the column."
restaurants.head()

,rest_name,addr,city,phone,cuisine_type
0,kokomo,6333 w. third st.,la,2139330773,american
1,feenix,8358 sunset blvd. west,hollywood,2138486677,american
2,parkway,510 s. arroyo pkwy .,pasadena,8187951001,californian
3,r-23,923 e. third st.,los angeles,2136877178,japanese
4,gumbo,6333 w. third st.,la,2139330358,cajun/creole


### The cutoff point

In [11]:
# Import process from fuzzywuzzy
from fuzzywuzzy import process

# Store the unique values of cuisine_type in unique_types
unique_types = restaurants.cuisine_type.unique()

# Calculate similarity of 'asian' to all values of unique_types
print("\nCalculate similarity of 'asian''")
print(process.extract('asian', unique_types, limit = len(unique_types)))

# Calculate similarity of 'american' to all values of unique_types
print("\nCalculate similarity of 'american'")
print(process.extract('american', unique_types, limit = len(unique_types)))

# Calculate similarity of 'italian' to all values of unique_types
print("\nCalculate similarity of 'italian'")
print(process.extract('italian', unique_types, limit = len(unique_types)))


Calculate similarity of 'asian''
[('asian', 100), ('indonesian', 72), ('italian', 67), ('russian', 67), ('american', 62), ('californian', 54), ('japanese', 54), ('mexican/tex-mex', 54), ('american ( new )', 54), ('mexican', 50), ('pizza', 40), ('cajun/creole', 36), ('diners', 36), ('middle eastern', 36), ('vietnamese', 36), ('pacific new wave', 36), ('fast food', 36), ('continental', 36), ('seafood', 33), ('chicken', 33), ('chinese', 33), ('hamburgers', 27), ('steakhouses', 25), ('southern/soul', 22), ('delis', 20), ('hot dogs', 18), ('coffee shops', 18), ('noodle shops', 18), ('health food', 18), ('eclectic', 18), ('coffeebar', 18), ('french ( new )', 18), ('desserts', 18)]

Calculate similarity of 'american'
[('american', 100), ('american ( new )', 90), ('mexican', 80), ('mexican/tex-mex', 68), ('asian', 62), ('californian', 53), ('italian', 53), ('russian', 53), ('middle eastern', 45), ('pacific new wave', 45), ('hamburgers', 44), ('indonesian', 44), ('chicken', 40), ('japanese', 3

given this data we should cut off the score at 80

### Remapping categories II

In [13]:
categories = ('italian', 'asian', 'american')

# Iterate through categories
for cuisine in categories:  
  # Create a list of matches, comparing cuisine with the cuisine_type column
  matches = process.extract(cuisine, restaurants['cuisine_type'], limit=len(restaurants.cuisine_type))

  # Iterate through the list of matches
  for match in matches:
     # Check whether the similarity score is greater than or equal to 80
    if match[1] >= 80:
      # If it is, select all rows where the cuisine_type is spelled this way, and set them to the correct cuisine
      restaurants.loc[restaurants['cuisine_type'] == match[0]] = cuisine
      
# Inspect the final result
print(restaurants['cuisine_type'].unique())

['american' 'californian' 'japanese' 'cajun/creole' 'hot dogs' 'diners'
 'delis' 'hamburgers' 'seafood' 'italian' 'coffee shops' 'russian'
 'steakhouses' 'mexican/tex-mex' 'noodle shops' 'middle eastern' 'asian'
 'vietnamese' 'health food' 'pacific new wave' 'indonesian' 'eclectic'
 'chicken' 'fast food' 'southern/soul' 'coffeebar' 'continental'
 'french ( new )' 'desserts' 'chinese' 'pizza']


# Generating Pairs

### Pairs of Restaurants

In [24]:
# install needed packages using pip
import sys
!{sys.executable} -m pip install recordlinkage

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [46]:
# Import package
import recordlinkage

# import additional dataset
restaurants_new = pd.read_csv("../datasets/restaurants_L2.csv")
restaurants_new.rename(columns={'type':'cuisine_type', 'name':'rest_name'}, inplace=True)
restaurants_new.head()

,rest_name,addr,city,phone,cuisine_type
0,arnie morton's of chicago,435 s. la cienega blv .,los angeles,3102461501,american
1,art's delicatessen,12224 ventura blvd.,studio city,8187621221,american
2,campanile,624 s. la brea ave.,los angeles,2139381447,american
3,fenix,8358 sunset blvd. west,hollywood,2138486677,american
4,grill on the alley,9560 dayton way,los angeles,3102760615,american


In [54]:
# drop the index that made it into the csv
try:
    restaurants_new = restaurants_new.drop('Unnamed: 0', axis=1)
except:
    "error while dropping the column."
restaurants_new.head()

,rest_name,addr,city,phone,cuisine_type
0,arnie morton's of chicago,435 s. la cienega blv .,los angeles,3102461501,american
1,art's delicatessen,12224 ventura blvd.,studio city,8187621221,american
2,campanile,624 s. la brea ave.,los angeles,2139381447,american
3,fenix,8358 sunset blvd. west,hollywood,2138486677,american
4,grill on the alley,9560 dayton way,los angeles,3102760615,american


In [58]:
# Create an indexer and object and find possible pairs
indexer = recordlinkage.Index()

# Block pairing on cuisine_type
indexer.block('cuisine_type')

# Generate pairs
pairs = indexer.index(restaurants, restaurants_new)
pairs

MultiIndex([( 0,   0),
            ( 0,   1),
            ( 0,   2),
            ( 0,   3),
            ( 0,   4),
            ( 0,   8),
            ( 0,   9),
            ( 0,  13),
            ( 0,  14),
            ( 0,  17),
            ...
            (55, 172),
            (55, 180),
            (55, 190),
            (55, 194),
            (55, 201),
            (55, 221),
            (55, 230),
            (55, 233),
            (55, 238),
            (55, 241)],
           length=3631)


### Similar restaurants

In [59]:
# Create a comparison object
comp_cl = recordlinkage.Compare()

# Find exact matches on city, cuisine_types - 
comp_cl.exact('city', 'city', label='city')
comp_cl.exact('cuisine_type', 'cuisine_type', label='cuisine_type')

# Find similar matches of rest_name
comp_cl.string('rest_name', 'rest_name', label='name', threshold = 0.8) 

# Get potential matches and print
potential_matches = comp_cl.compute(pairs, restaurants, restaurants_new)
print(potential_matches)

        city  cuisine_type  name
0  0       0             1   0.0
   1       0             1   0.0
   2       0             1   0.0
   3       0             1   0.0
   4       0             1   0.0
...      ...           ...   ...
55 221     1             1   0.0
   230     1             1   0.0
   233     1             1   0.0
   238     1             1   0.0
   241     1             1   0.0

[3631 rows x 3 columns]


In [60]:
# find matching pairs for all 3 columns
potential_matches[potential_matches.sum(axis = 1) >= 3]

,,city,cuisine_type,name
1,3,1,1,1.0
7,13,1,1,1.0
12,17,1,1,1.0
20,20,1,1,1.0
27,21,1,1,1.0
28,1,1,1,1.0
40,0,1,1,1.0
43,8,1,1,1.0
50,9,1,1,1.0
53,4,1,1,1.0


# Linking Dataframes

### Linking them together.

In [63]:
# Isolate potential matches with row sum >=3
matches = potential_matches[potential_matches.sum(axis=1) >= 3]

# Get values of second column index of matches
matching_indices = matches.index.get_level_values(1)

# Subset restaurants_new based on non-duplicate values
non_dup = restaurants_new[~restaurants_new.index.isin(matching_indices)]

# Append non_dup to restaurants
full_restaurants = restaurants.append(non_dup)
full_restaurants.head()

,rest_name,addr,city,phone,cuisine_type
0,kokomo,6333 w. third st.,la,2139330773,american
1,feenix,8358 sunset blvd. west,hollywood,2138486677,american
2,parkway,510 s. arroyo pkwy .,pasadena,8187951001,californian
3,r-23,923 e. third st.,los angeles,2136877178,japanese
4,gumbo,6333 w. third st.,la,2139330358,cajun/creole
